In [115]:
from glob import glob
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import cv2 from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import load_model

# Chargement des images


In [89]:
X = []
Y = []
for i in tqdm(glob('data/archive/train/Open_Eyes/*')):
    temp = cv2.resize(cv2.imread(i, cv2.IMREAD_GRAYSCALE), (128, 128))
    X.append(temp)
    Y.append(1)
for i in tqdm(glob('data/archive/train/Closed_Eyes/*')):
    temp = cv2.resize(cv2.imread(i, cv2.IMREAD_GRAYSCALE), (128, 128))
    X.append(temp)
    Y.append(0)


  0%|          | 0/2000 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [00:00<00:00, 3008.00it/s]


# Prétraitement (Normalisation)

In [123]:
for i in range(len(X)):
    X[i] = np.array(X[i])
    X[i] = (X[i] - np.min(X[i])) / (np.max(X[i]) - np.min(X[i]))
    X[i] /= 255.0
    X[i] = np.expand_dims(X[i], -1)

# Préparation des donnés d'entrainement et de test

In [98]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [100]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(3200, 128, 128, 1) (3200,) (800, 128, 128, 1) (800,)


# Définition du modèle CNN

In [109]:
model = Sequential([
# on précise le nombre de kernel = 32, la taille du kernel = 3 x 3
Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
MaxPooling2D(pool_size=(2, 2)),

Conv2D(64, (3, 3), activation='relu'),
MaxPooling2D(pool_size=(2, 2)),

Conv2D(64, (3, 3), activation='relu'),
MaxPooling2D(pool_size=(2, 2)),

Conv2D(128, (3, 3), activation='relu'),
MaxPooling2D(pool_size=(2, 2)),

Flatten(),
Dense(512, activation='relu'),
Dropout(0.5),
Dense(1, activation='sigmoid')
])
model.summary()

/home/wailk/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,489,921 (9.50 MB)

 Trainable params: 2,489,921 (9.50 MB)

 Non-trainable params: 0 (0.00 B)

In [110]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrainement du modèle

In [111]:
model.fit(X_train, Y_train, batch_size=32,  epochs=10)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 160ms/step - accuracy: 0.5253 - loss: 0.6900
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 18s 182ms/step - accuracy: 0.8529 - loss: 0.3240
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 21s 191ms/step - accuracy: 0.9691 - loss: 0.0963
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 196ms/step - accuracy: 0.9589 - loss: 0.1075
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 192ms/step - accuracy: 0.9757 - loss: 0.0751
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 19s 185ms/step - accuracy: 0.9761 - loss: 0.0476
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 19s 187ms/step - accuracy: 0.9769 - loss: 0.0655
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 19s 189ms/step - accuracy: 0.9897 - loss: 0.0239
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 19s 191ms/step - accuracy: 0.9910 - loss: 0.0248
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 19s 189ms/step - accuracy: 0.9933 - loss: 0.0132


# Evaluation de l'Accuracy du modèle sur les images tests

In [117]:
model.evaluate(X_test, Y_test)

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.9793 - loss: 0.0608


[0.03965805098414421, 0.9862499833106995]

# Sauvegarde du modèle

In [126]:
model.save("gg.keras")

# Fonction utilitaire pour détecter la position des yeux dans le visage

In [120]:
import cv2
import dlib

def predict(frame, model, detector, predictor):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Detect faces
    faces = detector(gray, 1)
    # Iterate over detected faces
    for face in faces:
        landmarks = predictor(gray, face)

        # Get the coordinates of the left and right eye
        left_eye = np.array([(landmarks.part(0).x, landmarks.part(0).y),
                            (landmarks.part(19).x, landmarks.part(19).y),
                            (landmarks.part(27).x, landmarks.part(27).y),
                            (landmarks.part(41).x, landmarks.part(41).y),
                            ], np.int32)

        right_eye = np.array([
                            (landmarks.part(27).x, landmarks.part(27).y),
                            (landmarks.part(24).x, landmarks.part(24).y),
                            (landmarks.part(16).x, landmarks.part(16).y),
                            (landmarks.part(46).x, landmarks.part(46).y),
                            ], np.int32)
        
        left_cropped_image = gray[left_eye[1][1]:left_eye[3][1]+15, left_eye[0][0]:left_eye[2][0]]
        left_cropped_image = cv2.resize(left_cropped_image, (128,128))
        left_cropped_image = (np.array(left_cropped_image) - np.min(left_cropped_image)) / (np.max(left_cropped_image) - np.min(left_cropped_image))
        left_cropped_image = left_cropped_image / 255.0

        right_cropped_image = gray[right_eye[1][1]:right_eye[3][1]+15, right_eye[0][0]:right_eye[2][0]]
        right_cropped_image = cv2.resize(right_cropped_image, (128,128))
        right_cropped_image = (np.array(right_cropped_image) - np.min(right_cropped_image)) / (np.max(right_cropped_image) - np.min(right_cropped_image))
        right_cropped_image = right_cropped_image / 255.0
        
        if model.predict(np.expand_dims(left_cropped_image,0))[0][0] < 0.5 or model.predict(np.expand_dims(right_cropped_image,0))[0][0] < 0.5:
            return 'closed'
        else:
            return 'opened' 
    
    return 'No Face'



# Lancement de la detection en streaming

In [125]:
# Open a video capture object
model = load_model("gg.keras")

# Load dlib's pre-trained model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

cap = cv2.VideoCapture(0)

try:
    while True:
        # Read a frame from the webcam
        ret, frame = cap.read()

        if not ret:
            break

        # Preprocess the frame to match your model's expected input format
        prediction = predict(frame, model, detector, predictor)
        if prediction == 'closed':
            cv2.putText(frame, 'Somnolant', (200, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        elif prediction == 'opened':
            cv2.putText(frame, 'Reveille', (200, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, 'Pas de visage detecte', (200, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        # Display the original frame using OpenCV
        cv2.imshow('Webcam', frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    # Release the video capture object
    cap.release()
    cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━